In [8]:
!pip install tweepy nltk scikit-learn matplotlib seaborn

In [9]:
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix

In [10]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


In [15]:
import zipfile
with zipfile.ZipFile("archive.zip",'r') as zip_ref:
  zip_ref.extractall("archive")

In [16]:
import os
os.listdir("archive")

['train_data.csv',
 'Exploring Data.ipynb',
 'test_data.csv',
 'vocab.py',
 'vocab.json']

In [17]:
import pandas as pd
df=pd.read_csv("archive/train_data.csv")
df.head()

,sentence,sentiment
0,awww that s a bummer you shoulda got david car...,0
1,is upset that he can t update his facebook by ...,0
2,i dived many times for the ball managed to sav...,0
3,my whole body feels itchy and like its on fire,0
4,no it s not behaving at all i m mad why am i h...,0


In [18]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [19]:
stop_words=set(stopwords.words('english'))
def clean_tweet(tweet):
  tweet=re.sub(r"@\w+|http\S+|#\w+"," ",tweet)
  tokens=word_tokenize(tweet.lower())
  tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
  return " ".join(tokens)
df["cleaned"]=df["sentence"].apply(clean_tweet)
df[["sentence","cleaned"]].head()

,sentence,cleaned
0,awww that s a bummer you shoulda got david car...,awww bummer shoulda got david carr third day
1,is upset that he can t update his facebook by ...,upset update facebook texting might cry result...
2,i dived many times for the ball managed to sav...,dived many times ball managed save rest go bounds
3,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,no it s not behaving at all i m mad why am i h...,behaving mad see


In [20]:
vectorizer=CountVectorizer()
X_train=vectorizer.fit_transform(df["cleaned"])
y_train=df["sentiment"]
model=MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [21]:
df1=pd.read_csv("archive/test_data.csv")
df1.head()
# X_test=vectorizer.transform(df1["sentence"])
# y_test=df1["sentiment"]

,sentence,sentiment
0,i loooooooovvvvvveee my kindle not that the dx...,1
1,reading my kindle love it lee childs is good read,1
2,ok first assesment of the kindle it fucking rocks,1
3,you ll love your kindle i ve had mine for a fe...,1
4,fair enough but i have the kindle and i think ...,1


In [22]:
# Clean the new test data (you must apply same cleaning as training!)
df1["cleaned"] = df1["sentence"].apply(clean_tweet)

# ✅ Use transform, NOT fit_transform
X_test = vectorizer.transform(df1["cleaned"])
y_test = df1["sentiment"]

# Predict
y_pred = model.predict(X_test)

In [23]:
print("Classification report:")
print(classification_report(y_test,y_pred))

Classification report:
              precision    recall  f1-score   support

           0       0.80      0.79      0.80       177
           1       0.80      0.81      0.80       182

    accuracy                           0.80       359
   macro avg       0.80      0.80      0.80       359
weighted avg       0.80      0.80      0.80       359



In [24]:
text = "Life is going okay"
cleaned = clean_tweet(text)
vectorized = vectorizer.transform([cleaned])
pred = model.predict(vectorized)

print("Predicted Sentiment:", "Positive" if pred[0] == 1 else "Negative")

Predicted Sentiment: Positive


In [25]:
def prediction_sentiment(tweet):
  cleaned=clean_tweet(tweet)
  vector=vectorizer.transform([cleaned])
  pred=model.predict(vector)[0]
  return f"Sentiment:{pred}"

In [26]:
import joblib
joblib.dump(model,'model.pkl')

['model.pkl']

In [27]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00


In [30]:
import gradio as gr
interface=gr.Interface(fn=prediction_sentiment,
                        inputs="text",
                        outputs="text",
                        title="Twitter Sentiment Analyzer",
                        description="Paste a tweet below to get its sentiment prediction")
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://56a893ac25afef4578.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
